In [1]:
import config
from utils import get_chromedriver, get_seduta


import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from bs4 import BeautifulSoup
from soup2dict import convert
import pandas as pd
import numpy as np
import xlsxwriter

In [2]:
CHROMEDRIVER_PATH = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\Tool Sedute Laurea\WebDriver\chromedriver"

In [3]:
GRADUATION_DATE = '03/10/2023'
ESSE3_URL = "https://studentionline.unipv.it/esse3/Home.do"
ROOT_URL = "https://studentionline.unipv.it/"    # used to open link inside esse3
CHECK_YEAR = '2013/2014'    # if anno iscrizione is before CHECK_YEAR, corresponding cell is filled in yellow

# Download Lista Studenti

In [79]:
# get seduta session
driver=get_seduta(CHROMEDRIVER_PATH, GRADUATION_DATE, ESSE3_URL, ROOT_URL, config)
wait=WebDriverWait(driver, 10)

# get commissione
time.sleep(1)
commissione=wait.until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[1]")))
commissione=convert(BeautifulSoup(commissione.get_attribute('outerHTML'), 'html.parser'))
comm_list=commissione['table'][0]['tbody'][0]['tr']
df_commissione=pd.DataFrame()
for i, com in enumerate(comm_list):
    df_commissione=pd.concat([df_commissione,
                              pd.DataFrame({'Docente': com['td'][0]['#text'], 'Ruolo': com['td'][1]['#text']}, index=[i+1])])
print(f'\n- Found {len(comm_list)} in commissione')
display(df_commissione)
file_name='Seduta Laurea '+'_'.join(GRADUATION_DATE.split('/')[::-1])+'_commissione.csv'
df_commissione.to_csv(file_name, index=False, sep=';')
print('\nFile saved in', file_name)
    
# get student list
if "footable-page-link" not in driver.page_source:     # single page
    try:
    #     driver.execute_script(f"window.scrollTo(0, {600});")   # scroll down to let the table appear
        time.sleep(1)
        studenti=wait.until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]")))
        studenti=convert(BeautifulSoup(studenti.get_attribute('outerHTML'), 'html.parser'))
        stud_list=studenti['table'][0]['tbody'][0]['tr']
        print(f'\n- Found {len(stud_list)} students')
    except:
        print('\n\n\n## Error for lista studenti pagina singola')
        raise
else:                                                  # multiple pages
    try:
        expected_rows = int(driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/span").get_attribute("textContent").split(' di ')[-1])
        num_pages = int(driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/span").get_attribute("textContent").split(',')[0].split('di')[-1])
        
        print(f'\n- Found {num_pages} pages')
        
        stud_list = []
        for i in range(num_pages):

            time.sleep(1)
            studenti=wait.until(
                    EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]")))
            studenti=convert(BeautifulSoup(studenti.get_attribute('outerHTML'), 'html.parser'))
            stud_list.extend(studenti['table'][0]['tbody'][0]['tr'])

            if i < (num_pages-1):
                driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/ul/li[5]/a").click()
                time.sleep(2)
                
        print(f'\n- Found {len(stud_list)} students')
        if len(stud_list) != expected_rows:
            print(f'\n\n\n## Error for lista studenti. Trovati {len(stud_list)}, attesi {expected_rows}')
    except:
        print('\n\n\n## Error for lista studenti pagine multiple')
        raise
        



- Found 7 in commissione


,Docente,Ruolo
1,GOTTARDO PIETRO,Membro Effettivo
2,OGLIARI MATTEO,Membro Effettivo
3,SALERNO FRANCESCO,Membro Effettivo
4,LEONE MARIA ANNA,Membro Effettivo
5,MAGLIACANI MICHELA,Membro Effettivo
6,BITETTO ALESSANDRO,Segretario
7,PICCOLI GABRIELE,Presidente



File saved in Seduta Laurea 03_10_2023_commissione.csv

- Found 2 pages



- Found 11 students


In [80]:
df_studenti=pd.DataFrame()
driver.switch_to.window(driver.window_handles[0])
for i, st in enumerate(stud_list):
    
    print(f"Reading student {i+1}/{len(stud_list)}", end = "\r")
    
    link=st['td'][10]['a'][0]['@href']
    driver.execute_script(f"window.open('{link}')")
    new_tab= driver.window_handles[i+1]
    driver.switch_to.window(new_tab)

    # get anno iscrizione
    iscrizione=wait.until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[1]")))
    iscrizione=convert(BeautifulSoup(iscrizione.get_attribute('outerHTML'), 'html.parser'))
    anno_iscrizione=[]
    for el in iscrizione['table'][0]['tbody'][0]['tr']:
        anno_iscrizione.append(el['#text'])

    # get punti tesi
    media=wait.until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/div[3]")))
    media=convert(BeautifulSoup(media.get_attribute('outerHTML'), 'html.parser'))
    crediti=media['div'][0]['dl'][0]['dd'][3]['#text'].replace('\xa0', '').replace('\u200b', '')
    crediti_tesi=media['div'][0]['dl'][0]['dd'][4]['#text'].replace('\xa0', '').replace('\u200b', '')
    media_pesata=media['div'][0]['dl'][0]['dd'][5]['#text'].replace('\xa0', '').replace('\u200b', '')

    # get voto proposto
    voto=wait.until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/div[4]")))
    voto=convert(BeautifulSoup(voto.get_attribute('outerHTML'), 'html.parser'))
    df_voto=pd.DataFrame(index=[i])
    for ind, el in enumerate(voto['div'][0]['dl'][0]['dt']):
        el_name=el['#text']
        el_val=voto['div'][0]['dl'][0]['dd'][ind]['navigablestring']
        el_val='\n'.join(el_val).replace('\xa0', '').replace('\u200b', '')
        if el_name == 'Voto proposto':
            el_val=int(el_val) 
        df_voto[el_name]=[el_val]
        
    add_row=pd.concat([pd.DataFrame({'Nome': st['td'][0]['#text'],
                                     'Matricola': st['td'][1]['#text'],
                                     'Nascita': st['td'][2]['#text'],
                                     'Corso': st['td'][3]['#text'],
                                     'Titolo tesi': st['td'][4]['#text'],
                                     'Relatore': st['td'][5]['#text'],
                                     'Anno iscrizione': '\n'.join(anno_iscrizione),
                                     'Crediti': crediti,
                                     'Crediti per tesi': crediti_tesi,
                                     'Media pesata': media_pesata.replace(' - ', '\n'),
                                     'link': link}, index=[i]),
                       df_voto], axis=1)

    df_studenti=pd.concat([df_studenti, add_row])
    df_studenti['Punti tesi']=''
    df_studenti['Lode']=''
    df_studenti['Encomio']=''
    df_studenti['Voto Finale']=0
    move_col = df_studenti.pop('link')
    df_studenti.insert(df_studenti.shape[1], 'link', move_col)
df_studenti

,Nome,Matricola,Nascita,Corso,Titolo tesi,Relatore,Anno iscrizione,Crediti,Crediti per tesi,Media pesata,...,Punteggio aggiuntivo,Motivazione punteggio aggiuntivo,Punteggio proposto,Motivazione punteggio proposto,Richiesta di eccellenza,Punti tesi,Lode,Encomio,Voto Finale,link
0,MEHDIPOOR KOOSHA,501765,03/05/1994,INTERNATIONAL BUSINESS AND ENTREPRENEURSHIP - ...,Migliorare l'efficienza educativa: un'applicaz...,PICCOLI GABRIELE,1 2021/2022 In corso\n2 2022/2023 In corso,99,24,93.378/110 \n25.467/30,...,0.0,Ottenuti 0.00 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
1,SCHWAGER TAMARA SOPHIE,524230,29/03/1997,INTERNATIONAL BUSINESS AND ENTREPRENEURSHIP - ...,Trusting the machine: An explorative study of ...,BARTOSIAK MARCIN LUKASZ,2 2022/2023 In corso\n2 2023/2024 Ripetente,108,24,102.457/110 \n27.943/30,...,1.2,Ottenuti 1.20 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
2,ARIOLA GABRIELE,503733,04/10/1999,ECONOMIA E LEGISLAZIONE D'IMPRESA,Sustainability and Reporting of a Smart City: ...,MAGLIACANI MICHELA,1 2021/2022 In corso\n2 2022/2023 In corso,102,24,105.701/110 \n28.828/30,...,0.9,Ottenuti 0.90 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
3,LATISANI DAVIDE,502796,11/11/1999,ECONOMIA E LEGISLAZIONE D'IMPRESA,La Sustainability Balanced Scorecard come stru...,MAGLIACANI MICHELA,1 2021/2022 In corso\n2 2022/2023 In corso,96,24,105.111/110 \n28.667/30,...,0.3,Ottenuti 0.30 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
4,ALBERTI LORENZO,503292,20/08/1999,"ECONOMICS, FINANCE AND INTERNATIONAL INTEGRATI...",Understanding poverty in Italy. A multidimensi...,LEONE MARIA ANNA,1 2021/2022 In corso\n2 2022/2023 In corso,96,24,105.86/110 \n28.871/30,...,0.3,Ottenuti 0.30 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
5,STANGA ELISABETTA,490064,18/07/1998,"ECONOMICS, FINANCE AND INTERNATIONAL INTEGRATI...",Food Insecurity. A focus on Sub-Saharan Africa,LEONE MARIA ANNA,1 2020/2021 In corso\n2 2021/2022 In corso\n2 ...,96,24,106.806/110 \n29.129/30,...,1.2,Ottenuti 1.20 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
6,MAZAL ROBIN,524243,21/05/2000,INTERNATIONAL BUSINESS AND ENTREPRENEURSHIP - ...,Initial Coin Offerings: Examining Potential Dr...,BITETTO ALESSANDRO,2 2022/2023 In corso\n2 2023/2024 Ripetente,99,24,104.729/110 \n28.563/30,...,0.6,Ottenuti 0.60 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
7,PELLEGRINO GIAN MARIA LUIGI GUSTAVO,501468,24/05/1999,ECONOMIA E LEGISLAZIONE D'IMPRESA,Elementi caratteristici e criticità della comp...,SALERNO FRANCESCO,1 2021/2022 In corso\n2 2022/2023 In corso,96,24,108.642/110 \n29.63/30,...,2.1,Ottenuti 2.10 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
8,SPIALTINI ALESSANDRO,490314,16/11/1996,ECONOMIA E LEGISLAZIONE D'IMPRESA,La composizione negoziale della crisi d'impres...,SALERNO FRANCESCO,1 2020/2021 In corso\n2 2021/2022 In corso\n2 ...,96,24,94.519/110 \n25.778/30,...,0.0,Ottenuti 0.00 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...
9,PIPERO ENEA,488396,10/07/1997,INTERNATIONAL BUSINESS AND ENTREPRENEURSHIP - ...,POWER PURCHASE AGREEMENTS NEL MERCATO ITALIANO...,OGLIARI MATTEO,1 2020/2021 In corso\n2 2021/2022 In corso\n2 ...,96,24,97.656/110 \n26.633/30,...,0.6,Ottenuti 0.60 punti per lodi;\nOttenuti in tot...,,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...


In [81]:
file_name='Seduta Laurea '+'_'.join(GRADUATION_DATE.split('/')[::-1])+'.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_studenti.to_excel(writer, sheet_name='Laurea', index=False)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Laurea']

wrap_format = workbook.add_format({'text_wrap' : True, 'align': 'top'})
top_format=workbook.add_format({'align': 'top'})
yellow_format=workbook.add_format({'text_wrap' : True, 'align': 'top', 'bg_color': 'yellow'})
worksheet.set_column(0, 5, 15, top_format)
worksheet.set_column(7, 9, 15, top_format)
worksheet.set_column(11, 11, 20, top_format)
worksheet.set_column(6, 6, 20, wrap_format)
worksheet.set_column(10, 10, 20, wrap_format)
worksheet.set_column(12, 12, 30, wrap_format)

# add formula for voto finale
col_num=df_studenti.columns.get_loc("Voto Finale")+1
col_letter_finale=chr(ord('@')+col_num)
col_num=df_studenti.columns.get_loc("Voto proposto")+1
col_letter_proposto=chr(ord('@')+col_num)
col_num=df_studenti.columns.get_loc("Punti tesi")+1
col_letter_punti=chr(ord('@')+col_num)
for ind, row in df_studenti.iterrows():
    worksheet.write_formula(col_letter_finale+str(ind+2), '='+ col_letter_proposto+str(ind+2) +' + '+ col_letter_punti+str(ind+2))
    # https://xlsxwriter.readthedocs.io/working_with_formulas.html
    
# fill cell with yellow if "Anno iscrizione" is before CHECK_YEAR
col_num=df_studenti.columns.get_loc("Anno iscrizione")+1
col_letter=chr(ord('@')+col_num)
for ind, row in df_studenti.iterrows():
    anno=[]
    for an in row['Anno iscrizione'].split('\n'):
        anno.append([int(x.split('/')[0]) for i, x in enumerate(an.split(' ')) if '/' in x][0])
    if min(anno) <= int(CHECK_YEAR.split('/')[0]):
        worksheet.write(col_letter+str(ind+2), row['Anno iscrizione'], yellow_format)

writer.save()

print('\nFile saved in', file_name)


File saved in Seduta Laurea 03_10_2023.xlsx


C:\Users\Alessandro Bitetto\AppData\Local\Temp\ipykernel_17556\1437380815.py:42: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


# Upload Voti



Prima compila la colonna `Punti tesi` e `Lode` nel file excel

In [4]:
# df_commissione=pd.read_csv('Seduta Laurea '+'_'.join(GRADUATION_DATE.split('/')[::-1])+'_commissione.csv', sep=';')
df_studenti=pd.read_excel('Seduta Laurea '+'_'.join(GRADUATION_DATE.split('/')[::-1])+'.xlsx')
df_studenti['Lode']=df_studenti['Lode'].fillna('')
df_studenti['Encomio']=df_studenti['Encomio'].fillna('')

In [32]:
# get seduta session
driver=get_seduta(CHROMEDRIVER_PATH, GRADUATION_DATE, ESSE3_URL, ROOT_URL, config)
wait=WebDriverWait(driver, 10)

In [6]:
driver.switch_to.window(driver.window_handles[0])

for i, row in df_studenti.iterrows():
    
    print(f"Uploading student {i+1}/{len(df_studenti)}", end = "\r")
    
    link=ROOT_URL+row['link']
    driver.execute_script(f"window.open('{link}')")
    new_tab= driver.window_handles[i+1]
    driver.switch_to.window(new_tab)
    
    # input punti tesi
    try:
        punti_tesi=wait.until(
                        EC.presence_of_element_located((By.ID, 'grad-dettLau-puntiTesi')))
        punti_tesi.click()
        punti_tesi.clear()
        punti_tesi.send_keys(row['Punti tesi'])
        wait.until(EC.presence_of_element_located((By.ID, 'grad-dettLau-annotazioni'))).click()
    except:
        print(f'\n## Error for punti tesi {row["Nome"]} ({i+1})')

    # tick Lode and Encomio
    if row['Lode'] != '':
        try:
            lode=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-lode1')))
            lode.click()
        except:
            print(f'\n## Error for Lode {row["Nome"]} ({i+1})')
    if row['Encomio'] != '':
        try:
            encomio=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-encomio1')))
            encomio.click()
        except:
            print(f'\n## Error for Encomio {row["Nome"]} ({i+1})')

    # tick Commissione
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        commissione=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/form/div[3]")))
        commissione=convert(BeautifulSoup(commissione.get_attribute('outerHTML'), 'html.parser'))
        get_checkbox_id=commissione['div'][0]['div'][0]['@id'].replace('gbox', 'cb')
        all_commissione=wait.until(
                        EC.presence_of_element_located((By.ID, get_checkbox_id)))
        all_commissione.click()
    except:
        print(f'\n## Error for Commissione {row["Nome"]} ({i+1})')

    # save
    try:
        save=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-btnSubmit')))
        save.click()
    except:
        print(f'\n## Error for Save {row["Nome"]} ({i+1})')

    save and exit
    try:
        save_exit=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-btnSubmitExit')))
        save_exit.click()
    except:
        print(f'\n## Error for Save and Exit {row["Nome"]} ({i+1})')


# check final grade mismatch
driver.switch_to.window(driver.window_handles[0])

if "footable-page-link" not in driver.page_source:     # single page
    final_grade=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]")))
    final_grade=convert(BeautifulSoup(final_grade.get_attribute('outerHTML'), 'html.parser'))
    elem_list=final_grade['table'][0]['tbody'][0]['tr']
else:                                                  # multiple pages
    expected_rows = int(driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/span").get_attribute("textContent").split(' di ')[-1])
    num_pages = int(driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/span").get_attribute("textContent").split(',')[0].split('di')[-1])
    
    print(f'\n- Found {num_pages} pages')
        
    elem_list=[]
    for i in range(num_pages):
        
        time.sleep(1)
        final_grade=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]")))
        final_grade=convert(BeautifulSoup(final_grade.get_attribute('outerHTML'), 'html.parser'))
        elem_list.extend(final_grade['table'][0]['tbody'][0]['tr'])

        if i < (num_pages-1):
            driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/ul/li[5]/a").click()
            time.sleep(2)
                
    print(f'- Found {len(elem_list)} students')
    if len(elem_list) != expected_rows:
        print(f'\n\n\n## Error for lista studenti. Trovati {len(elem_list)}, attesi {expected_rows}')
        
for i, el in enumerate(elem_list):
    grade=el['td'][8]['#text']
    lode = True if 'L' in grade else False
    grade_evaluated=int(grade.replace('L', ''))
    matricola=int(el['td'][1]['#text'])
    name=el['td'][0]['#text']
    grade_expected=min(df_studenti[df_studenti['Matricola'] == matricola]['Voto Finale'].values[0], 110)
    lode_expected = True if df_studenti[df_studenti['Matricola'] == matricola]['Lode'].values[0] != '' else False
    if grade_evaluated != grade_expected or lode != lode_expected:
        print(f'\n## Expected Votazione Finale mismatch: {name} ({i+1})')
        
print('\nVotazione Finale: Check Done')

Uploading student 9/9
Votazione Finale: Check Done
